<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Template_Prompt_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Gerador Automatizado de Petições Jurídicas
Este projeto implementa um sistema automatizado para geração de petições jurídicas. O sistema funciona através de uma equipe de agentes especializados que trabalham em conjunto para transformar dados estruturados em uma petição jurídica formatada.

Funcionalidade Principal:
- O sistema opera através de quatro agentes especializados:
Leitor de YAML: Processa os dados estruturados da petição
Leitor de Template: Analisa o modelo base da petição
Escritor: Combina os dados com o template para criar a petição
Revisor: Garante a qualidade e precisão do documento final
- O processo transforma automaticamente um arquivo YAML contendo os dados da petição em um documento jurídico formatado em Markdown, seguindo um template predefinido.
- Resultado
O sistema gera uma petição jurídica completa e formatada, pronta para uso, salvando o resultado final em um arquivo markdown. Este sistema foi desenvolvido para automatizar e padronizar a criação de petições jurídicas, mantendo a qualidade e precisão necessárias em documentos legais.


In [ ]:
!pip install --q crewai
!pip install --q 'crewai[tools]'

<img src="https://raw.githubusercontent.com/sandeco/crewai/main/images/2.png" width="80%"
/>

In [ ]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import FileReadTool
from litellm import completion
import os
import yaml

# Ferramenta para leitura de arquivos YAML e templates
reader_tool = FileReadTool()

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Inicializar o modelo OpenAI
llm = LLM (
    model = 'gemini/gemini-1.5-flash',
    temperature=0.2
    )


# Definindo os agentes
leitor_yaml = Agent(
    role='Leitor de Dados',
    goal='Ler os dados do arquivo {yaml} de uma petição. Esses dados servirão de apoio para'
         'o escritor redigir a petição apropriadamente e com os dados corretos.'
         'Crie uma lista de seções que devem ser usadas na petição e passe '
         'para o escritor.',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um especialista em leitura e interpretação de dados legais, "
        "com vasta experiência em processar informações de documentos legais."
    ),
    tools=[reader_tool],
    allow_delegation=False,
    llm=llm
)

leitor_template = Agent(
    role='Leitor de Templates',
    goal='Ler o arquivo de template {template} de uma petição '
         'para que o escritor possa escrever uma petição baseada '
         'no template. Somente leia e passe o template para o escritor',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um especialista em leitura de templates e "
        "possui vasta experiência em processar informações "
        "de documentos legais."
    ),
    tools=[reader_tool],
    allow_delegation=False,
    llm=llm
)

escritor = Agent(
    role='Escritor de Petições',
    backstory=
        "Você é um advogado altamente qualificado e experiente "
        "na elaboração de peças jurídicas precisas e persuasivas. "
        "É reconhecido por sua capacidade de articular argumentos "
        "convincente que evidenciam os méritos de cada caso, "
        "garantindo que os interesses de seus clientes sejam representados "
        "de forma eficaz e justa. Você tem a grande habilidade "
        "de transformar dados brutos em belíssimas peças jurídicas."
        "Alterando templates deixando de acordo com todos os dados"
        "da petição."
    ,
    goal=
        f"""Você receberá os dados do agente leitor de YAML,
        que devem ser usados como insumo para a escrita
        de uma nova petição. Todos os dados do YAML devem
        estar no texto. Não deixe faltar nenhuma informação
        proveniente do YAML na petição. Os textos de todas
        as informações das seções, como partes, fatos,
        fundamentação jurídica, pedido, provas ou
        qualquer outra seção que venha do YAML,
        devem conter o título da seção e o texto da seção
        deve ser escrito em parágrafos corridos,
        sem pontos ou listas.
        Use o template do Leitor de Templates com o referência para criar o texto
        """
    ,
    verbose=True,
    memory=True,
    llm=llm,
    allow_delegation=False
)

revisor = Agent(
    role='Revisor de Petições',
    goal='Você revisará a petição escrita para garantir precisão e clareza. '
         'Estando tudo correto, salve todo o texto revisado da petição.',
    verbose=True,
    memory=True,
    backstory=(
        "Um revisor meticuloso com um olhar aguçado para detalhes, "
        "garantindo que cada petição esteja livre de erros e bem formulada."
    ),
    llm=llm,
    allow_delegation=False
)

# Definindo as tarefas
leitura_dados = Task(
    description=(
        "Use a ferramenta de leitura para ler o arquivo {yaml} e extrair os dados necessários para a petição."
    ),
    expected_output='Dados estruturados da petição prontos para uso.',
    agent=leitor_yaml,
)

leitura_template = Task(
    description=(
        "Use a ferramenta de leitura para ler o arquivo {template} "
        "que servirá de referência para a petição. "
    ),
    expected_output='Template petição pronta para uso pelo escritor.',
    agent=leitor_template,
)

escrita_peticao = Task(
    description=(
        "Escreva em Markdown uma petição baseada nos dados do YAML. "
        "Substitua as seções do template pelos dados do YAML e adicione novas seções se necessário."
    ),
    expected_output=(
        'Uma petição bem escrita e estruturada em Markdown.'
    ),
    agent=escritor,
    context=[leitura_dados, leitura_template]
)

revisao_peticao = Task(
    description=(
        "Revise a petição escrita em Markdown para garantir precisão e clareza. "
        "Estando tudo correto, salve todo o texto revisado da petição. "
        "Retire ```markdown."
    ),
    expected_output='Um arquivo de texto em Markdown com a '
                    'petição revisada e pronta para ser submetida. '
                    'É muito importante que o texto não contenha '
                    'coisas como "```markdown" e "```".',
    agent=revisor,
    output_file='peticao.md',
    context=[escrita_peticao]
)

# Configurando o Crew
crew = Crew(
    agents=[leitor_yaml, leitor_template, escritor, revisor],
    tasks=[leitura_dados, leitura_template, escrita_peticao, revisao_peticao],
    process=Process.sequential,
    cache=True
)

inputs = {'yaml':'peticao.yaml',
          'template':'template.md'}

result = crew.kickoff(inputs=inputs)

print(result)


In [12]:
from IPython.display import Markdown
Markdown(result.raw)

# **Petição Inicial**

## **Endereçamento**

**Tribunal:** Tribunal de Justiça do Estado de Goiás
**Vara:** 1ª Vara Cível
**Comarca:** Goiânia
**Processo:** 04458320853480
**Juiz:** José Justo

## **Qualificação das Partes**

### **Autor**

- **Nome completo:** João do Pé de Feijão
- **Nacionalidade:** Brasileiro
- **Estado Civil:** Solteiro
- **Profissão:** Pedreiro
- **RG:** MG-12.345.678
- **CPF:** 123.456.789-00
- **Endereço completo:** Rua das Sentido, 123, Centro, Goiânia, GO, 74000-000

### **Réu**

- **Nome completo:** Maria de Blablabla
- **Nacionalidade:** Brasileira
- **Estado Civil:** Casada
- **Profissão:** Advogada
- **RG:** GO-98.765.432
- **CPF:** 987.654.321-00
- **Endereço completo:** Avenida do Chão de Ouro, 456, Setor Oeste, Goiânia, GO, 74000-001

### **Advogado**

- **Nome:** Dr. Sandeco Macedo
- **OAB:** OAB/GO 12345
- **Endereço profissional:** Alameda dos Anjos, 789, Setor Sul, Goiânia, GO, 74000-002
- **Telefone:** (62) 1234-5678
- **Email:** sandeco@advocacia_doidera.com.br

## **Fatos**

Em 20 de maio de 2023, na Praça Central de Goiânia, o Autor, João do Pé de Feijão, foi vítima de ofensas verbais proferidas pelo Réu, Maria de Blablabla. O Réu, de forma gratuita e sem qualquer justificativa, chamou o Autor de "mizeravi", causando-lhe grande constrangimento e humilhação.

## **Danos Morais**

Em decorrência das ofensas sofridas, o Autor experimentou abalo emocional significativo, com prejuízo em suas relações pessoais e profissionais. A conduta do Réu, além de violar a dignidade do Autor, causou-lhe sofrimento psicológico, necessitando de acompanhamento profissional para lidar com as sequelas do evento.

## **Fundamentação Jurídica**

A conduta do Réu configura ato ilícito, conforme disposto no **Art. 186 do Código Civil**, que prevê a responsabilidade civil por danos morais. O **Art. 927 do Código Civil** estabelece que aquele que, por ação ou omissão voluntária, negligência ou imprudência, violar direito e causar dano a outrem, ainda que exclusivamente moral, comete ato ilícito.

A jurisprudência pátria reconhece o direito à indenização por danos morais em casos como o presente, em que a ofensa à dignidade da pessoa humana é evidente. O **princípio da dignidade da pessoa humana**, consagrado na Constituição Federal, garante a proteção da honra, da imagem e da integridade moral de cada indivíduo.

## **Pedidos**

Diante do exposto, requer-se:

1. **Citação:** Do Réu, para que, querendo, apresente contestação, no prazo legal.
2. **Procedência:** Da presente ação, condenando o Réu ao pagamento de indenização por danos morais, a ser arbitrada por este Juízo, levando em consideração a gravidade do dano sofrido pelo Autor.
3. **Valor da causa:** R$ 50.000,00 (cinquenta mil reais).
4. **Outros pedidos:**
    - A condenação do Réu ao pagamento de juros de mora e correção monetária, a partir do evento danoso.
    - A condenação do Réu ao pagamento das custas processuais e honorários advocatícios, estes fixados em 20% (vinte por cento) do valor da condenação.

## **Provas**

Para comprovar os fatos alegados, o Autor se oferece a produzir as seguintes provas:

### **Documentais**

- Boletim de Ocorrência registrado na data do evento.
- Relatórios médicos que comprovam o abalo emocional sofrido pelo Autor.

### **Testemunhais**

- **Nome:** Ana Aninha Ana
- **Endereço:** Rua das Palmeiras, 321, Centro, Goiânia, GO
- **Nome:** Pedro Pedreira Noventa
- **Endereço:** Avenida das Américas, 654, Setor Bueno, Goiânia, GO

### **Periciais**

- Perícia psicológica para avaliação dos danos emocionais sofridos pelo Autor.

## **Medidas Antecipatórias**

Requer-se, ainda, a concessão de medida liminar, consistente na **proibição de contato do Réu com o Autor**, em qualquer meio de comunicação, inclusive por telefone, e-mail, redes sociais e mensagens instantâneas.

A presente medida se justifica pela necessidade de evitar novas ofensas e constrangimentos ao Autor, garantindo sua segurança e integridade moral. A urgência da medida decorre do risco de dano irreparável ao Autor, caso o Réu continue a importuná-lo.

## **Encerramento**

Nestes termos, pede deferimento.

**Goiânia, 30 de julho de 2024.**

### **Assinatura**

**Advogado:** Dr. Sandeco Macedo
**OAB:** OAB/GO 12345